# Feature Engineering Front End (FEFE)

In [4]:
from scoring.fefe import notebook_front_end, FeatureEngineeringAPI

# Data import

_Do not forget to cast the datetimes!_

In [5]:
import pandas as pd
import sys
sys.path.insert(0, '../scoring')
data = pd.read_csv('../scoring/fefe_core/tests/toy_data.csv', index_col=0)


# cast your datetimes, please!
data['TIME'] = pd.to_datetime(data['TIME'])
data['TRANSACTION_TIME'] = pd.to_datetime(data['TRANSACTION_TIME'])

data.head()

,CUSTOMER_ID,TIME,HOME_PLACE,TRANSACTION_AMOUNT,TRANSACTION_CLASS,TRANSACTION_FEE,TRANSACTION_PLACE,TRANSACTION_PURPOSE,TRANSACTION_TIME,TRANSACTION_TYPE,ID_TRANSACTION
0,0,2015-01-01 00:23:52,Sázava,999.0,ATM,99.413752,Sázava,cash,2014-01-02 04:32:51,DEBIT,772
1,0,2015-01-01 00:23:52,Sázava,784.0,ACCOUNT,110.171181,NaN,NaN,2014-01-08 06:22:34,CREDIT,4529
2,0,2015-01-01 00:23:52,Sázava,575.0,ACCOUNT,21.029130,NaN,travelling,2014-01-09 05:05:51,DEBIT,5093
3,0,2015-01-01 00:23:52,Sázava,1140.0,POS,45.302900,Skalice,hazard,2014-01-10 03:45:41,DEBIT,5666
4,0,2015-01-01 00:23:52,Sázava,1515.0,ACCOUNT,32.438470,NaN,NaN,2014-01-14 11:45:52,CREDIT,8379


# Front End (for new features)

This part should ease config creation.

However, it is not needed - see next part how to modify config.

In [6]:
notebook_front_end(data, port=8062)

Dash is running on http://127.0.0.1:8062/

 * Serving Flask app "scoring.fefe_core.front_end.app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8062/ (Press CTRL+C to quit)


# Config (output from front end)

The config is made from several parts. If you are not able to open the front end, you can still manipulate with the config manually.  

## Config description (tl;dr: look at the example)


<details>
<summary>
Definition of individual parts of the config and individual parameters.  

___
**\~\~\~ Click to expand \~\~\~**
___
</summary>


### meta:

_Definition of global informations, to be applied anywhere._

- granularity
    - describes the granularity to be used for aggregation.
    - possible values:
        - 'days' (or 'd', 'day', etc.)
        - 'weeks'
        - 'months'
        - 'years
        - 'order' 
            - takes into account only the order, not the time.
            - useful to get last transaction amount (order = 1)
            - useful for calculation using `NUM_GROUP` variables
- index
    - name of the index column in the dataset
    - usually 'CUSTOMER_ID', 'SKP_CREDIT_CASE', 'CUID' ...
- inf_value
    - value to replace infinities, if they appear in data
- nan_value
    - value to replace nans, if they appear in data
- target_time
    - time column, which is used as decision time.
- transaction_time
    - time column, to be used as time of the individual transaction.
- order
    - specification of column to be used for subsetting of the data using time ranges.
    - 'TIME_ORDER' means that column with such name will be created from `target_time` and `transaction_time`
    - otherwise FEFE will use this specified column.

### simple:

_Definition of simple variables._

- VARIABLE_NAME:
    - name of varibale, on which following transformations will be applied.
    - transformations:
        - functions
            - list of functions
            - usually, `pandas.DataFrame.GroupBy.[fun]` is applied
        - segmentations
            - list of variable names or `str(None)` (None, but as a string)
            - values from such variables are used in pd.DataFrame.query like `VARIABLE_NAME == 'value'`
        - time_ranges
            - list of tuples (which are saved as strings)
            - single tuple means from-to, which will be used with `order` column
        - queries
            - string to be put into `pd.DataFrame.query()` function
            - possible inputs:
                - `VARIABLE == 'value'` (notice double `=`)
                - `VARIABLE < 20`
                - `VARIABLE_1 == 'value_1';VARIABLE_1 == 'value_2'`
                    - this will iterate over individual queries
                - ```VARIABLE_1 == 'value_1';VARIABLE_1 =='value_2';VARIABLE_2 == 'value_a';VARIABLE_2 == 'value_b'```
                - ...

### ratio:

_Definition of ratio variables_

- DENOMINATOR_VARIABLE_NAME
    - NUMERATOR_VARIABLE_NAME
        - transformations:
            - similar like in simple case
            - functions are defined as tuple of functions
            - time_ranges are tuples of tuples of integers (two time ranges together)
            - segmentations & queries are applied at the variables at the same time

### time_since:

- TIME_VARIABLE_NAME
    - transformations:
        - from - whether take first or last event
        - segmentations  - same as in simple
        - queries - same as in simple
          
</details>

## Config example

In [7]:
CONFIG={
    'meta': {
        'granularity': 'days',
        'index': 'CUSTOMER_ID',
        'target_time': 'TIME',
        'transaction_time': 'TRANSACTION_TIME',
        'inf_value': None,
        'nan_value': None,
        'order': 'TIME_ORDER',

    },
    'ratio': {
        'TRANSACTION_AMOUNT': {
            'TRANSACTION_AMOUNT': {
                'functions': ["('min', 'min')", "('max', 'max')", "('mean', 'mean')", "('sum', 'sum')"],
                'segmentations': ['None', 'TRANSACTION_PURPOSE'],
                'time_ranges': ['((0, 30), (30, 180))', '((0, 180), (180, 360))'],
            },
        },
    },
    'simple': {
        'TRANSACTION_AMOUNT': {
            'functions': ['min', 'max', 'sum', 'mean'],
            'queries': "TRANSACTION_FEE < 100;TRANSACTION_CLASS == 'ATM'",
            'segmentations': ['None', 'TRANSACTION_PURPOSE', 'TRANSACTION_TYPE', 'TRANSACTION_PLACE'],
            'time_ranges': ['(0, 360)', '(0, inf)'],
        },
        'TRANSACTION_FEE': {
            'functions': ['min', 'max', 'sum', 'mean'],
            'queries': "TRANSACTION_FEE < 100;TRANSACTION_CLASS == 'ATM'",
            'segmentations': ['None', 'TRANSACTION_PURPOSE', 'TRANSACTION_TYPE'],
            'time_ranges': ['(0, 360)', '(0, inf)'],
        },
        'TRANSACTION_PURPOSE': {
            'functions': ['mode', 'mode_multicolumn'],
            'segmentations': ['None'],
            'time_ranges': ['(0, 360)'],
        }
    },
    'time_since': {'TRANSACTION_TIME': {'from': ['first', 'last'], 'queries': "TRANSACTION_PURPOSE=='hazard'"}},
}


# Initialisation of calculation

In [8]:
name_shortening_dictionary = {
    "TRANSACTION_FEE":"FEE",
    "TRANSACTION_AMOUNT": "AMNT",
    "TRAVELLING": "TRVL",
    "HAZARD": "HZRD",
    "TRANSACTION_PURPOSE": "PRPS",
    "TRANSACTION": "TNX",
    "_EQ_": "_",
    "_0D_": "_",
    "_INFD":"",   
}

fe = FeatureEngineeringAPI(
    config=CONFIG,
    shortening_dictionary=name_shortening_dictionary,
    logger_kwargs={"log_level": 20})

2020-12-09 11:53:40,904 - scoring.fefe_core.utils.logger - INFO - Logger created.


# Calculation of dataframe

In [9]:
from IPython.display import display
with pd.option_context('display.max_columns', 200):
    
    display(fe.dataframe(data, max_nan_share=0.9))

2020-12-09 11:53:42,770 - scoring.fefe_core.utils.logger - INFO - 
            Calculating SIMPLE features
            _________________________
            
2020-12-09 11:53:43,154 - scoring.fefe_core.utils.logger - WARNING - Missing config part: queries. Using default value: 
2020-12-09 11:53:43,189 - scoring.fefe_core.utils.logger - INFO - Number of simple features (before pruning): 258
2020-12-09 11:53:43,191 - scoring.fefe_core.utils.logger - INFO - # of omitted features: 1
2020-12-09 11:53:43,193 - scoring.fefe_core.utils.logger - INFO - Number of simple features (after pruning): 258
2020-12-09 11:53:43,283 - scoring.fefe_core.utils.logger - INFO - 
            Calculating RATIO features
            _________________________
            
2020-12-09 11:53:43,285 - scoring.fefe_core.utils.logger - WARNING - Missing config part: queries. Using default value: 
2020-12-09 11:53:43,287 - scoring.fefe_core.utils.logger - INFO - Calculating TRANSACTION_AMOUNT / TRANSACTION_AMOUNT
2020-12

,MIN_AMNT_360D_FEE_LT_100,MAX_AMNT_360D_FEE_LT_100,SUM_AMNT_360D_FEE_LT_100,MEAN_AMNT_360D_FEE_LT_100,MIN_AMNT_FEE_LT_100,MAX_AMNT_FEE_LT_100,SUM_AMNT_FEE_LT_100,MEAN_AMNT_FEE_LT_100,MIN_AMNT_360D_TNX_CLASS_ATM,MAX_AMNT_360D_TNX_CLASS_ATM,SUM_AMNT_360D_TNX_CLASS_ATM,MEAN_AMNT_360D_TNX_CLASS_ATM,MIN_AMNT_TNX_CLASS_ATM,MAX_AMNT_TNX_CLASS_ATM,SUM_AMNT_TNX_CLASS_ATM,MEAN_AMNT_TNX_CLASS_ATM,MIN_AMNT_360D,MAX_AMNT_360D,SUM_AMNT_360D,MEAN_AMNT_360D,MIN_AMNT,MAX_AMNT,SUM_AMNT,MEAN_AMNT,MIN_AMNT_360D_PRPS_CASH,MAX_AMNT_360D_PRPS_CASH,SUM_AMNT_360D_PRPS_CASH,MEAN_AMNT_360D_PRPS_CASH,MIN_AMNT_PRPS_CASH,MAX_AMNT_PRPS_CASH,SUM_AMNT_PRPS_CASH,MEAN_AMNT_PRPS_CASH,MIN_AMNT_360D_PRPS_NULL,MAX_AMNT_360D_PRPS_NULL,SUM_AMNT_360D_PRPS_NULL,MEAN_AMNT_360D_PRPS_NULL,MIN_AMNT_PRPS_NULL,MAX_AMNT_PRPS_NULL,SUM_AMNT_PRPS_NULL,MEAN_AMNT_PRPS_NULL,MIN_AMNT_360D_PRPS_TRVL,MAX_AMNT_360D_PRPS_TRVL,SUM_AMNT_360D_PRPS_TRVL,MEAN_AMNT_360D_PRPS_TRVL,MIN_AMNT_PRPS_TRVL,MAX_AMNT_PRPS_TRVL,SUM_AMNT_PRPS_TRVL,MEAN_AMNT_PRPS_TRVL,MIN_AMNT_360D_PRPS_HZRD,MAX_AMNT_360D_PRPS_HZRD,SUM_AMNT_360D_PRPS_HZRD,MEAN_AMNT_360D_PRPS_HZRD,MIN_AMNT_PRPS_HZRD,MAX_AMNT_PRPS_HZRD,SUM_AMNT_PRPS_HZRD,MEAN_AMNT_PRPS_HZRD,MIN_AMNT_360D_PRPS_GROCERIES,MAX_AMNT_360D_PRPS_GROCERIES,SUM_AMNT_360D_PRPS_GROCERIES,MEAN_AMNT_360D_PRPS_GROCERIES,MIN_AMNT_PRPS_GROCERIES,MAX_AMNT_PRPS_GROCERIES,SUM_AMNT_PRPS_GROCERIES,MEAN_AMNT_PRPS_GROCERIES,MIN_AMNT_360D_PRPS_CHARITY,MAX_AMNT_360D_PRPS_CHARITY,SUM_AMNT_360D_PRPS_CHARITY,MEAN_AMNT_360D_PRPS_CHARITY,MIN_AMNT_PRPS_CHARITY,MAX_AMNT_PRPS_CHARITY,SUM_AMNT_PRPS_CHARITY,MEAN_AMNT_PRPS_CHARITY,MIN_AMNT_360D_PRPS_LUXURY,MAX_AMNT_360D_PRPS_LUXURY,SUM_AMNT_360D_PRPS_LUXURY,MEAN_AMNT_360D_PRPS_LUXURY,MIN_AMNT_PRPS_LUXURY,MAX_AMNT_PRPS_LUXURY,SUM_AMNT_PRPS_LUXURY,MEAN_AMNT_PRPS_LUXURY,MIN_AMNT_360D_TNX_TYPE_DEBIT,MAX_AMNT_360D_TNX_TYPE_DEBIT,SUM_AMNT_360D_TNX_TYPE_DEBIT,MEAN_AMNT_360D_TNX_TYPE_DEBIT,MIN_AMNT_TNX_TYPE_DEBIT,MAX_AMNT_TNX_TYPE_DEBIT,SUM_AMNT_TNX_TYPE_DEBIT,MEAN_AMNT_TNX_TYPE_DEBIT,MIN_AMNT_360D_TNX_TYPE_CREDIT,MAX_AMNT_360D_TNX_TYPE_CREDIT,SUM_AMNT_360D_TNX_TYPE_CREDIT,MEAN_AMNT_360D_TNX_TYPE_CREDIT,MIN_AMNT_TNX_TYPE_CREDIT,MAX_AMNT_TNX_TYPE_CREDIT,SUM_AMNT_TNX_TYPE_CREDIT,MEAN_AMNT_TNX_TYPE_CREDIT,MIN_AMNT_360D_TNX_PLACE_SÁZAVA,MAX_AMNT_360D_TNX_PLACE_SÁZAVA,SUM_AMNT_360D_TNX_PLACE_SÁZAVA,MEAN_AMNT_360D_TNX_PLACE_SÁZAVA,...,MIN_FEE_PRPS_HZRD,MAX_FEE_PRPS_HZRD,SUM_FEE_PRPS_HZRD,MEAN_FEE_PRPS_HZRD,MIN_FEE_360D_PRPS_GROCERIES,MAX_FEE_360D_PRPS_GROCERIES,SUM_FEE_360D_PRPS_GROCERIES,MEAN_FEE_360D_PRPS_GROCERIES,MIN_FEE_PRPS_GROCERIES,MAX_FEE_PRPS_GROCERIES,SUM_FEE_PRPS_GROCERIES,MEAN_FEE_PRPS_GROCERIES,MIN_FEE_360D_PRPS_CHARITY,MAX_FEE_360D_PRPS_CHARITY,SUM_FEE_360D_PRPS_CHARITY,MEAN_FEE_360D_PRPS_CHARITY,MIN_FEE_PRPS_CHARITY,MAX_FEE_PRPS_CHARITY,SUM_FEE_PRPS_CHARITY,MEAN_FEE_PRPS_CHARITY,MIN_FEE_360D_PRPS_LUXURY,MAX_FEE_360D_PRPS_LUXURY,SUM_FEE_360D_PRPS_LUXURY,MEAN_FEE_360D_PRPS_LUXURY,MIN_FEE_PRPS_LUXURY,MAX_FEE_PRPS_LUXURY,SUM_FEE_PRPS_LUXURY,MEAN_FEE_PRPS_LUXURY,MIN_FEE_360D_TNX_TYPE_DEBIT,MAX_FEE_360D_TNX_TYPE_DEBIT,SUM_FEE_360D_TNX_TYPE_DEBIT,MEAN_FEE_360D_TNX_TYPE_DEBIT,MIN_FEE_TNX_TYPE_DEBIT,MAX_FEE_TNX_TYPE_DEBIT,SUM_FEE_TNX_TYPE_DEBIT,MEAN_FEE_TNX_TYPE_DEBIT,MIN_FEE_360D_TNX_TYPE_CREDIT,MAX_FEE_360D_TNX_TYPE_CREDIT,SUM_FEE_360D_TNX_TYPE_CREDIT,MEAN_FEE_360D_TNX_TYPE_CREDIT,MIN_FEE_TNX_TYPE_CREDIT,MAX_FEE_TNX_TYPE_CREDIT,SUM_FEE_TNX_TYPE_CREDIT,MEAN_FEE_TNX_TYPE_CREDIT,MODE_PRPS_360D,MODE_MULTICOLUMN_PRPS_360D_0,MODE_MULTICOLUMN_PRPS_360D_1,MODE_MULTICOLUMN_PRPS_360D_2,MIN_AMNT_30D_DIV_MIN_AMNT_30D_180D,MAX_AMNT_30D_DIV_MAX_AMNT_30D_180D,MEAN_AMNT_30D_DIV_MEAN_AMNT_30D_180D,SUM_AMNT_30D_DIV_SUM_AMNT_30D_180D,MIN_AMNT_180D_DIV_MIN_AMNT_180D_360D,MAX_AMNT_180D_DIV_MAX_AMNT_180D_360D,MEAN_AMNT_180D_DIV_MEAN_AMNT_180D_360D,SUM_AMNT_180D_DIV_SUM_AMNT_180D_360D,MIN_AMNT_30D_PRPS_CASH_DIV_MIN_AMNT_30D_180D_PRPS_CASH,MAX_AMNT_30D_PRPS_CASH_DIV_MAX_AMNT_30D_180D_PRPS_CASH,MEAN_AMNT_30D_PRPS_CASH_DIV_MEAN_AMNT_30D_180D_PRPS_CASH,SUM_AMNT_30D_PRPS_CASH

# Output in sql

In [10]:
print(fe.sql(data,
             table_name='data',
             feature_subset=[
#                  'MAX_TRANSACTION_AMOUNT_0D_INFD',
#                  'MIN_TRANSACTION_AMOUNT_0D_INFD'
             ]))

2020-12-09 11:54:00,633 - scoring.fefe_core.utils.logger - INFO - 
            Creating SQL for SIMPLE features
            _________________________
            
2020-12-09 11:54:00,634 - scoring.fefe_core.utils.logger - INFO - Creating features in sql
2020-12-09 11:54:00,640 - scoring.fefe_core.utils.logger - WARNING - Missing config part: queries. Using default value: 
2020-12-09 11:54:00,641 - scoring.fefe_core.utils.logger - INFO - 
            Creating SQL for RATIO features
            _________________________
            
2020-12-09 11:54:00,643 - scoring.fefe_core.utils.logger - WARNING - Missing config part: queries. Using default value: 
2020-12-09 11:54:00,646 - scoring.fefe_core.utils.logger - INFO - Creating features in sql
2020-12-09 11:54:00,650 - scoring.fefe_core.utils.logger - INFO - Creating features in sql
2020-12-09 11:54:00,662 - scoring.fefe_core.utils.logger - INFO - Number of created ratio variables: 64
2020-12-09 11:54:00,664 - scoring.fefe_core.utils.logger


            select t.*, 
                   ceil(TIME - TRANSACTION_TIME) as TIME_ORDER,
              from _TABLENAME_ t
             where 1=1
            
            
            

            -- MIN_AMNT_360D_FEE_LT_100
            select
            CUSTOMER_ID,
            
            min(case when TIME_ORDER >= 0
                         and TIME_ORDER <= 360
                         and TRANSACTION_FEE < 100
                        then TRANSACTION_AMOUNT
                        end)
             as MIN_AMNT_360D_FEE_LT_100
            from data
            group by CUSTOMER_ID
            order by CUSTOMER_ID;
            

            -- MIN_AMNT_360D_TNX_CLASS_ATM
            select
            CUSTOMER_ID,
            
            min(case when TIME_ORDER >= 0
                         and TIME_ORDER <= 360
                         and TRANSACTION_CLASS = 'ATM'
                        then TRANSACTION_AMOUNT
                        end)
             as MIN_AMNT_360D_TNX_C